In [1]:
from pathlib import Path
from time import time
import pandas as pd
from videosentimentanalysis.usecases.download_video_use_case import VideoUseCase
from videosentimentanalysis.adapters.logging.custom_logger import CustomLogger
from videosentimentanalysis.adapters.logging.loguru_logger import LoguruLogger
from videosentimentanalysis.adapters.extract_audio.moviepy_extraction import MoviePyAudioExtractor
from videosentimentanalysis.usecases.video_utility_use_case import VideoUtilityUseCase
from videosentimentanalysis.adapters.transcribe_audio.speechrecognition import SpeechRecognition
from videosentimentanalysis.adapters.perform_sentiment_analysis.textblob_analysis import TextBlobAnalysis
from videosentimentanalysis.adapters.translate_text.google_translate import TextTranslation
from videosentimentanalysis.usecases.protocols.translate_text import LanguageOptions
from videosentimentanalysis.adapters.extract_emotions.text2emotion_extractor import GetEmotions
from videosentimentanalysis.domain.audio import Audio
import json
from threading import Thread
from multiprocessing import Process, Manager


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Constants

In [2]:
OUTPUT_FOLDER= Path("./output/")

EXPLAIN HERE THE ARCHITECTURE OF THE PROJECT

To run this project you need to have the following installed:
- ffmpeg
- python3.12
- poetry

## Setup Adapters
### Logging


In [3]:
custom_logger = CustomLogger(Path("./download_log.txt"))
loguru_logger = LoguruLogger(Path("./loguru_logger.log"))

### Video Utilities

In [4]:

moviepy_extractor = MoviePyAudioExtractor(Path("./tmp_audio"))
speechrecognition = SpeechRecognition()
sentimentanalysis = TextBlobAnalysis()
text_translation = TextTranslation()
emotion_extraction = GetEmotions()

Load all the Youtube URLs to a Pydantic Model called RawURLVideo

In [5]:
video_use_case = VideoUseCase(video_output_directory=Path("tmp_videos"), logger=custom_logger)

In [6]:
raw_url_videos=video_use_case.load_raw_url_video(Path('./video_urls.txt'))
raw_url_videos

[RawVideoUrl(url='https://www.youtube.com/watch?v=G025oxyWv0E\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=G7xFfezsJ1s\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=qAsdnGJFbt8\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=9dpwVc6Olo0\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=PtjKDyLMtio\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=ah_WSxzN8v4\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=qRdzw2Osl8o\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=LkWQvzrv6gI\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=2P1nE57dvNw\n'),
 RawVideoUrl(url='https://www.youtube.com/watch?v=vvkIF0NlIzA')]

In [7]:
sequential_time_taken = time()
video_use_case.download_raw_videos_sequentially(raw_url_videos)
sequential_time_taken = time() - sequential_time_taken

2024-06-23 20:35:46.709609+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-23 20:35:47.990163+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/3c9e25db-b386-46ce-9bdc-08e169cf2ca5.mp4 {} ()
2024-06-23 20:35:48.077421+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-23 20:35:49.003831+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/dd19d765-c109-4304-b2e0-44323bae1b49.mp4 {} ()
2024-06-23 20:35:49.097828+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()
2024-06-23 20:35:50.239737+00:00 INFO: Downloaded video from url='https://www.you

In [8]:
threading_time_taken = time()
video_use_case.download_raw_videos_threading(raw_url_videos)
threading_time_taken = time() - threading_time_taken

2024-06-23 20:36:04.326776+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' on thread 1 {} ()2024-06-23 20:36:04.326594+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=G025oxyWv0E\n' on thread 0 {} ()

2024-06-23 20:36:04.328427+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=NkC9AiJf7gI\n' on thread 4 {} ()
2024-06-23 20:36:04.327050+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' on thread 2 {} ()
2024-06-23 20:36:04.327174+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=N2xk9nSEgxk\n' on thread 3 {} ()
2024-06-23 20:36:04.436937+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()
2024-06-23 20:36:04.441904+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=9dgmp3KmwGg\n' {} ()
2024-06-23 20:36:04.444166+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=NkC9A

In [9]:
process_time_taken = time()
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)
process_time_taken = time() - process_time_taken

2024-06-23 20:36:11.360259+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' on process 2 {} ()
2024-06-23 20:36:11.500098+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()
2024-06-23 20:36:12.827236+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/02eeba65-ced5-474f-99ab-9c39f9cd40ad.mp4 {} ()
2024-06-23 20:36:11.398090+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=qAsdnGJFbt8\n' on process 6 {} ()
2024-06-23 20:36:11.504582+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=qAsdnGJFbt8\n' {} ()
2024-06-23 20:36:12.940341+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=qAsdnGJFbt8\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/f5b62c0c-e8e6-4e77-b4

## Results of each method of downloading videos and time taken

In [10]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [sequential_time_taken, threading_time_taken, process_time_taken]
})

,Method,Time Taken
0,Sequential,17.708158
1,Threading,6.851484
2,Multiprocessing,5.229064


Blah Blah Blah Monkey stuff

In [11]:

for video_index in range(len(videos)):
    video=videos[video_index]
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    
    video_output_folder= OUTPUT_FOLDER / video.title
    video_output_folder.mkdir(exist_ok=True, parents=True)
    
    video.move(video_output_folder)
    print(video.local_storage_path)
    
    audio = video_utlity_use_case.extract_audio()
    audio.move(video_output_folder)
    
    with open(video_output_folder / "sentimental_analysis.json", mode="w") as sentimental_analysis_file:
        sentimental_analysis_file.write(json.dumps(video_utlity_use_case.get_sentiment_analysis()))
    
    with open(video_output_folder / "transcribe_text.txt", mode="w") as transcribe_text_file:
        transcribe_text_file.write(video_utlity_use_case.transcribe_audio())
    
    with open(video_output_folder / "translate_text.txt", mode="w") as translate_text_file:
        translate_text_file.write(video_utlity_use_case.translate_text(source_lang=LanguageOptions.ENGLISH, target_lang=LanguageOptions.SPANISH))
    
    with open(video_output_folder / "detect_emotions.json", mode="w") as detect_emotions_file:
        detect_emotions_file.write(json.dumps(video_utlity_use_case.detect_emotions()))
    


output/If Educational Videos Were Filmed Like Music Videos/22a45982-153a-4ec8-8017-7ce90f417182.mp4
2024-06-23 20:36:16.412747+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} ()
MoviePy - Writing audio in tmp_audio/7a8ca4d8-774a-4ee0-ba58-cf23c88c4dfc.wav


MoviePy - Done.
2024-06-23 20:36:17.020823+00:00 INFO: Returning cached audio for video If Educational Videos Were Filmed Like Music Videos {} ()
2024-06-23 20:36:17.020916+00:00 INFO: Transcribing audio from video If Educational Videos Were Filmed Like Music Videos {} ()
2024-06-23 20:36:27.858673+00:00 INFO: Performing sentiment analysis on video If Educational Videos Were Filmed Like Music Videos {} ()
2024-06-23 20:36:27.878485+00:00 INFO: Returning cached transcribed audio for video If Educational Videos Were Filmed Like Music Videos {} ()
2024-06-23 20:36:27.878663+00:00 INFO: Returning cached transcribed audio for video If Educational Videos Were Filmed Like Music Videos {} ()
2024-06-23 20:36:28.144723+00:00 INFO: Returning cached transcribed audio for video If Educational Videos Were Filmed Like Music Videos {} ()
output/Casually Explained: Guide to College and University/fb71e132-ed68-4bf4-b466-90b216bd2d96.mp4
2024-06-23 20:36:29.608661+00:00 INFO: Extracting audio from vide

MoviePy - Done.
2024-06-23 20:36:30.401296+00:00 INFO: Returning cached audio for video Casually Explained: Guide to College and University {} ()
2024-06-23 20:36:30.401406+00:00 INFO: Transcribing audio from video Casually Explained: Guide to College and University {} ()
2024-06-23 20:36:46.382682+00:00 INFO: Performing sentiment analysis on video Casually Explained: Guide to College and University {} ()
2024-06-23 20:36:46.388043+00:00 INFO: Returning cached transcribed audio for video Casually Explained: Guide to College and University {} ()
2024-06-23 20:36:46.388579+00:00 INFO: Returning cached transcribed audio for video Casually Explained: Guide to College and University {} ()
2024-06-23 20:36:46.514859+00:00 INFO: Returning cached transcribed audio for video Casually Explained: Guide to College and University {} ()
output/Potter Puppet Pals: The Mysterious Ticking Noise/02eeba65-ced5-474f-99ab-9c39f9cd40ad.mp4
2024-06-23 20:36:48.339464+00:00 INFO: Extracting audio from video P

MoviePy - Done.
2024-06-23 20:36:48.950805+00:00 INFO: Returning cached audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
2024-06-23 20:36:48.950919+00:00 INFO: Transcribing audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
2024-06-23 20:36:52.764531+00:00 INFO: Performing sentiment analysis on video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
2024-06-23 20:36:52.765811+00:00 INFO: Returning cached transcribed audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
2024-06-23 20:36:52.766027+00:00 INFO: Returning cached transcribed audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
2024-06-23 20:36:53.017535+00:00 INFO: Returning cached transcribed audio for video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
output/Fastest Rap EVER/ba561456-78da-4aeb-b8bd-1341d330d90d.mp4
2024-06-23 20:36:53.098927+00:00 INFO: Extracting audio from video Fastest Rap EVER {} ()
MoviePy - Writing audio in t

MoviePy - Done.
2024-06-23 20:36:53.562425+00:00 INFO: Returning cached audio for video Fastest Rap EVER {} ()
2024-06-23 20:36:53.562532+00:00 INFO: Transcribing audio from video Fastest Rap EVER {} ()
2024-06-23 20:36:59.823376+00:00 INFO: Performing sentiment analysis on video Fastest Rap EVER {} ()
2024-06-23 20:36:59.825686+00:00 INFO: Returning cached transcribed audio for video Fastest Rap EVER {} ()
2024-06-23 20:36:59.825867+00:00 INFO: Returning cached transcribed audio for video Fastest Rap EVER {} ()
2024-06-23 20:37:01.165068+00:00 INFO: Returning cached transcribed audio for video Fastest Rap EVER {} ()
output/THE PARTS OF A CELL SONG | Science Music Video/3e3c0351-1ca4-46b7-b7f4-e0fb4f5730ce.mp4
2024-06-23 20:37:01.760781+00:00 INFO: Extracting audio from video THE PARTS OF A CELL SONG | Science Music Video {} ()
MoviePy - Writing audio in tmp_audio/152daf36-bcbd-4268-a151-8c98179abcea.wav


...

MoviePy - Done.
2024-06-23 20:38:27.614109+00:00 INFO: Returning cached audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
2024-06-23 20:38:27.614193+00:00 INFO: Transcribing audio from video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
2024-06-23 20:38:34.683728+00:00 INFO: Performing sentiment analysis on video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
2024-06-23 20:38:34.686315+00:00 INFO: Returning cached transcribed audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
2024-06-23 20:38:34.686522+00:00 INFO: Returning cached transcribed audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
2024-06-23 20:38:36.197041+00:00 INFO: Returning cached transcribed audio for video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
output/Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim/7832d791-3735-4cbe-8d9a-0685242bc4af.

MoviePy - Done.
2024-06-23 20:38:37.456477+00:00 INFO: Returning cached audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
2024-06-23 20:38:37.456578+00:00 INFO: Transcribing audio from video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
2024-06-23 20:38:44.946038+00:00 INFO: Performing sentiment analysis on video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
2024-06-23 20:38:44.948651+00:00 INFO: Returning cached transcribed audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
2024-06-23 20:38:44.948843+00:00 INFO: Returning cached transcribed audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
2024-06-23 20:38:45.087327+00:00 INFO: Returning cached transcribed audio for video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
output/Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Ent

MoviePy - Done.
2024-06-23 20:38:46.252133+00:00 INFO: Returning cached audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
2024-06-23 20:38:46.252242+00:00 INFO: Transcribing audio from video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
2024-06-23 20:38:51.100630+00:00 INFO: Performing sentiment analysis on video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
2024-06-23 20:38:51.102237+00:00 INFO: Returning cached transcribed audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
2024-06-23 20:38:51.102450+00:00 INFO: Returning cached transcribed audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
2024-06-23 20:38:51.335933+00:00 INFO: Returning cached transcribed audio for video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
output/The Kai N' Speed Show | Of

MoviePy - Done.
2024-06-23 20:38:51.873156+00:00 INFO: Returning cached audio for video The Kai N' Speed Show | Official Trailer {} ()
2024-06-23 20:38:51.873285+00:00 INFO: Transcribing audio from video The Kai N' Speed Show | Official Trailer {} ()
2024-06-23 20:39:05.122837+00:00 INFO: Performing sentiment analysis on video The Kai N' Speed Show | Official Trailer {} ()
2024-06-23 20:39:05.124876+00:00 INFO: Returning cached transcribed audio for video The Kai N' Speed Show | Official Trailer {} ()
2024-06-23 20:39:05.125085+00:00 INFO: Returning cached transcribed audio for video The Kai N' Speed Show | Official Trailer {} ()
2024-06-23 20:39:06.404702+00:00 INFO: Returning cached transcribed audio for video The Kai N' Speed Show | Official Trailer {} ()
output/Futurama - The End of the Universe/c2f4943d-b47f-4485-b039-cf98450628d9.mp4
2024-06-23 20:39:06.736660+00:00 INFO: Extracting audio from video Futurama - The End of the Universe {} ()
MoviePy - Writing audio in tmp_audio/107

MoviePy - Done.
2024-06-23 20:39:07.481674+00:00 INFO: Returning cached audio for video Futurama - The End of the Universe {} ()
2024-06-23 20:39:07.481811+00:00 INFO: Transcribing audio from video Futurama - The End of the Universe {} ()
2024-06-23 20:39:14.171109+00:00 INFO: Performing sentiment analysis on video Futurama - The End of the Universe {} ()
2024-06-23 20:39:14.173512+00:00 INFO: Returning cached transcribed audio for video Futurama - The End of the Universe {} ()
2024-06-23 20:39:14.173826+00:00 INFO: Returning cached transcribed audio for video Futurama - The End of the Universe {} ()
2024-06-23 20:39:14.319112+00:00 INFO: Returning cached transcribed audio for video Futurama - The End of the Universe {} ()


### Sequential time for extract audio

In [12]:
videos=video_use_case.download_raw_videos_multiprocessing(raw_url_videos)

2024-06-23 20:39:14.998928+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=G025oxyWv0E\n' on process 0 {} ()
2024-06-23 20:39:15.145162+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' {} ()
2024-06-23 20:39:16.372508+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=G025oxyWv0E\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/92ce8caf-3a79-40fd-a886-6636187f5ac9.mp4 {} ()
2024-06-23 20:39:15.051063+00:00 INFO: Downloading url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' on process 2 {} ()
2024-06-23 20:39:15.177728+00:00 INFO: Bypassed age restriction for video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' {} ()
2024-06-23 20:39:16.403011+00:00 INFO: Downloaded video from url='https://www.youtube.com/watch?v=Tx1XIm6q4r4\n' saved to /Users/dariavesela/VideoSentimentAnalysis/videosentimentanalysis/services/tmp_videos/c0d6d434-c5bc-4653-a0

In [13]:
extract_audio_time_seq = time()
for video in videos:
    video_utlity_use_case=VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    video_utlity_use_case.extract_audio()

extract_audio_time_seq = time() -extract_audio_time_seq
print(f"Time taken for sequential extraction is {extract_audio_time_seq} seconds")

2024-06-23 20:39:19.864148+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} ()
MoviePy - Writing audio in tmp_audio/dd0d0607-dfc9-4217-81dc-6f0e9d435de3.wav


MoviePy - Done.
2024-06-23 20:39:20.626793+00:00 INFO: Extracting audio from video Casually Explained: Guide to College and University {} ()
MoviePy - Writing audio in tmp_audio/b211e696-ca3f-4414-ba75-d975e2ab4fc3.wav


MoviePy - Done.
2024-06-23 20:39:21.542217+00:00 INFO: Extracting audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
MoviePy - Writing audio in tmp_audio/bd6d0436-7350-42fd-b1b3-2fceb9ae20a7.wav


MoviePy - Done.
2024-06-23 20:39:22.210067+00:00 INFO: Extracting audio from video Fastest Rap EVER {} ()
MoviePy - Writing audio in tmp_audio/307e137a-9677-466e-8ef4-71cb3614f9c2.wav


MoviePy - Done.
2024-06-23 20:39:22.727063+00:00 INFO: Extracting audio from video THE PARTS OF A CELL SONG | Science Music Video {} ()
MoviePy - Writing audio in tmp_audio/c208de04-97ff-4dfd-8a7b-029ce0f2ecda.wav


...

MoviePy - Done.
2024-06-23 20:39:26.989915+00:00 INFO: Extracting audio from video Rick and Morty | S6E6 Cold Open: The Dinosaurs Return | adult swim {} ()
MoviePy - Writing audio in tmp_audio/bd245918-4952-44bd-9ce3-965f9219f707.wav


MoviePy - Done.
2024-06-23 20:39:27.644587+00:00 INFO: Extracting audio from video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
MoviePy - Writing audio in tmp_audio/c676b9db-7ecf-4180-9855-2e531b34cecd.wav


MoviePy - Done.
2024-06-23 20:39:28.234865+00:00 INFO: Extracting audio from video The Kai N' Speed Show | Official Trailer {} ()
MoviePy - Writing audio in tmp_audio/194e5538-be33-4a1b-9fb3-89fb848e9878.wav


MoviePy - Done.
2024-06-23 20:39:28.755433+00:00 INFO: Extracting audio from video Futurama - The End of the Universe {} ()
MoviePy - Writing audio in tmp_audio/128bf4c2-9319-4eec-a1ea-df031b024380.wav


MoviePy - Done.
Time taken for sequential extraction is 9.636855125427246 seconds


### Threading time for extract audio

In [14]:
def threading_wrapper_extractor(results: list[Audio],index: int,video_utlity_use_case :  VideoUtilityUseCase):
    custom_logger.log(f"Processing {video_utlity_use_case.video.title} on thread {index}")
    results[index]=video_utlity_use_case.extract_audio()

In [15]:
extract_audio_time_threading = time()
results = [None] * len(videos)
threads = []
for index, video in enumerate(videos):
    video_utlity_use_case= VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    thread = Thread(target=threading_wrapper_extractor, args=(results,index,video_utlity_use_case))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()
extract_audio_time_threading = time() -extract_audio_time_threading
print(f"Time taken for threading extraction is {extract_audio_time_threading} seconds")

2024-06-23 20:39:29.506075+00:00 INFO: Processing If Educational Videos Were Filmed Like Music Videos on thread 0 {} ()
2024-06-23 20:39:29.506432+00:00 INFO: Processing Casually Explained: Guide to College and University on thread 1 {} ()
2024-06-23 20:39:29.506536+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} ()
2024-06-23 20:39:29.506617+00:00 INFO: Processing Potter Puppet Pals: The Mysterious Ticking Noise on thread 2 {} ()
2024-06-23 20:39:29.506701+00:00 INFO: Extracting audio from video Casually Explained: Guide to College and University {} ()
2024-06-23 20:39:29.539261+00:00 INFO: Processing Fastest Rap EVER on thread 3 {} ()
2024-06-23 20:39:29.506909+00:00 INFO: Extracting audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
2024-06-23 20:39:29.540786+00:00 INFO: Extracting audio from video Fastest Rap EVER {} ()
2024-06-23 20:39:29.565485+00:00 INFO: Processing THE PARTS OF A CELL SONG | Science Music Video

chunk:   0%|          | 0/2798 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in tmp_audio/13ec11a2-4712-4590-9363-e7b81e01058d.wav


MoviePy - Writing audio in tmp_audio/5573c7b5-2d08-4c69-9216-ad381b620732.wav


chunk:   0%|          | 5/2798 [00:00<00:40, 68.63it/s, now=None]

chunk:   0%|          | 0/5303 [00:00<?, ?it/s, now=None]

...

MoviePy - Done.













chunk:  99%|█████████▉| 4031/4068 [00:09<00:00, 1226.42it/s, now=None]










                                                                      




                                                       

                                                                






                                                 




chunk:  87%|████████▋ | 4028/4635 [00:09<00:00, 1126.08it/s, now=None]

chunk:  77%|███████▋  | 4057/5303 [00:09<00:01, 1175.73it/s, now=None]






chunk:  85%|████████▌ | 4124/4834 [00:09<00:00, 1174.43it/s, now=None]




chunk:  87%|████████▋ | 4053/4635 [00:09<00:00, 1270.23it/s, now=None]






chunk:  87%|████████▋ | 4182/4834 [00:09<00:00, 1342.48it/s, now=None]

chunk:  78%|███████▊  | 4147/5303 [00:09<00:00, 1376.23it/s, now=None]




chunk:  93%|█████████▎| 4330/4635 [00:09<00:00, 1717.15it/s, now=None]






chunk:  92%|█████████▏| 4458/4834 [00:09<00:00, 1756.60it/s, now=None]

chunk:  83%|████████▎ | 4411/5303 [00:09<00:00, 1752.7

MoviePy - Done.







chunk:  99%|█████████▉| 4590/4635 [00:09<00:00, 1970.99it/s, now=None]






chunk:  98%|█████████▊| 4743/4834 [00:09<00:00, 2072.89it/s, now=None]




                                                                      

                                                                






                                                 

chunk:  88%|████████▊ | 4692/5303 [00:09<00:00, 2067.45it/s, now=None]






chunk:  98%|█████████▊| 4759/4834 [00:09<00:00, 2072.89it/s, now=None]

chunk:  88%|████████▊ | 4692/5303 [00:09<00:00, 2067.45it/s, now=None]






                                                                      

                                                                

chunk:  90%|█████████ | 4788/5303 [00:09<00:00, 2067.45it/s, now=None]

                                                                      

MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
Time taken for threading extraction is 11.463006973266602 seconds


### Processing time for extract audio

In [16]:
extract_audio_time_processes = time()
manager = Manager()
results = manager.list([None] * len(videos))
processes = []
for index, video in enumerate(videos):
    video_utlity_use_case= VideoUtilityUseCase(video=video, logger=custom_logger, extract_audio=moviepy_extractor, extract_text=speechrecognition, extract_polarity_and_sensitivity=sentimentanalysis, translate=text_translation,extract_emotions=emotion_extraction)
    process = Process(target=video_utlity_use_case._multiprocessing_wrapper_audio_extraction_wrapper, args=(results,index))
    processes.append(process)
    process.start()
for process in processes:
    process.join()
extract_audio_time_processes = time() -extract_audio_time_processes
print(f"Time taken for processing extraction is {extract_audio_time_processes} seconds")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already 

2024-06-23 20:39:42.823208+00:00 INFO: Extracting audio using process 0 {} ()
2024-06-23 20:39:42.823350+00:00 INFO: Extracting audio from video If Educational Videos Were Filmed Like Music Videos {} ()
MoviePy - Writing audio in tmp_audio/b2f7602b-67f7-4384-a8fe-1e50e5f549a8.wav
2024-06-23 20:39:42.832005+00:00 INFO: Extracting audio using process 1 {} ()
2024-06-23 20:39:42.832141+00:00 INFO: Extracting audio from video Casually Explained: Guide to College and University {} ()
MoviePy - Writing audio in tmp_audio/465219c3-a96a-4aa5-89ef-2c2794293e3e.wav
2024-06-23 20:39:42.936265+00:00 INFO: Extracting audio using process 2 {} ()
2024-06-23 20:39:42.936394+00:00 INFO: Extracting audio from video Potter Puppet Pals: The Mysterious Ticking Noise {} ()
MoviePy - Writing audio in tmp_audio/48d39209-1f05-4691-be3a-272a1685d1ca.wav
2024-06-23 20:39:42.915345+00:00 INFO: Extracting audio using process 3 {} ()
2024-06-23 20:39:42.915488+00:00 INFO: Extracting audio from video Fastest Rap EVE

chunk:   5%|▌         | 204/4068 [00:00<00:02, 1828.26it/s, now=None][nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dariavesela/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dariavesela/nltk_data...
chunk:  28%|██▊       | 459/1627 [00:00<00:00, 4548.70it/s, now=None][nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dariavesela/nltk_data...
chunk:  14%|█▍        | 485/3426 [00:00<00:00, 4849.75it/s, now=None]

2024-06-23 20:39:43.048510+00:00 INFO: Extracting audio using process 5 {} ()
2024-06-23 20:39:43.048639+00:00 INFO: Extracting audio from video THE ROCK CYCLE SONG | Science Music Video {} ()
MoviePy - Writing audio in tmp_audio/dd4abf62-a17b-43e4-bcdb-63971212934c.wav
2024-06-23 20:39:43.088773+00:00 INFO: Extracting audio using process 9 {} ()
2024-06-23 20:39:43.089896+00:00 INFO: Extracting audio from video Rick and Jerry Measure Reality | Rick and Morty | adult swim {} ()
MoviePy - Writing audio in tmp_audio/5fe29b80-399c-40cd-8ac8-e1e061a7dba9.wav
2024-06-23 20:39:43.073005+00:00 INFO: Extracting audio using process 4 {} ()
2024-06-23 20:39:43.073130+00:00 INFO: Extracting audio from video THE PARTS OF A CELL SONG | Science Music Video {} ()
MoviePy - Writing audio in tmp_audio/80331b2c-b89a-405d-9099-2d3f0b592166.wav
2024-06-23 20:39:43.109262+00:00 INFO: Extracting audio using process 6 {} ()
2024-06-23 20:39:43.112566+00:00 INFO: Extracting audio from video L.O - Don't wanna 

2024-06-23 20:39:43.370983+00:00 INFO: Extracting audio using process 11 {} ()
2024-06-23 20:39:43.371110+00:00 INFO: Extracting audio from video Pirates of the Caribbean - The Curse of the Black Pearl - Jack's Entrance {} ()
MoviePy - Writing audio in tmp_audio/0d3664bb-44d0-44dd-a8d3-5ff500835c38.wav
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.


chunk:  43%|████▎     | 2058/4834 [00:00<00:00, 3370.53it/s, now=None]

MoviePy - Done.
MoviePy - Done.


MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.


chunk:  96%|█████████▌| 4630/4834 [00:01<00:00, 5826.31it/s, now=None]

MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
MoviePy - Done.
Time taken for processing extraction is 3.6286582946777344 seconds


### Comparison of seqential , threading , processing

In [17]:
pd.DataFrame({
    "Method": ["Sequential", "Threading", "Multiprocessing"],
    "Time Taken": [extract_audio_time_seq, extract_audio_time_threading, extract_audio_time_processes]
})

,Method,Time Taken
0,Sequential,9.636855
1,Threading,11.463007
2,Multiprocessing,3.628658
